<a href="https://colab.research.google.com/github/INT2-19/INT2_PyTorch-CrashCourse/blob/main/CharlesCovNet_Colour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 250
batch_size = 64
learning_rate = 0.001

# Convolutional neural network
class ConvNet(nn.Module):
    def __init__(self, num_classes=102):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(25088, 2048)
        self.fc2 = nn.Linear(2048, num_classes)
        self.drop = nn.Dropout(0.4)
        # Batch normalisation used on convolution layers, dropout 20% used on fully connected linear layers

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.pool(out)
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.pool(out)
        out = F.relu(self.bn3(self.conv3(out)))
        out = self.pool(out)
        out = F.relu(self.bn4(self.conv4(out)))
        out = self.pool(out)
        out = F.relu(self.bn5(self.conv5(out)))
        out = self.pool(out)
        out = out.view(out.size(0), -1)
        out = F.relu(self.drop(self.fc1(out)))
        out = self.fc2(out)
        return out


# Create model and push to device
model = ConvNet().to(device)

# Get loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define transforms with data augmentation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomVerticalFlip(p=0.05),
    transforms.RandomHorizontalFlip(p=0.1),
    transforms.RandomRotation(45),
    transforms.ColorJitter(contrast=0.25, saturation=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Flower102: Unknown number and various sized color images in 102 classes, with 40 to 258 images per class
train_dataset = torchvision.datasets.Flowers102(root='./data', split='train',
                                                download=True, transform=train_transform)
test_dataset = torchvision.datasets.Flowers102(root='./data', split='test',
                                               download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                          shuffle=False)

def train(model, train_loader, criterion, optimizer, device, epoch, smoothing=0.1):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Apply label smoothing
        num_classes = model.fc2.out_features
        one_hot_labels = torch.zeros(labels.size(0), num_classes).to(device)
        one_hot_labels.scatter_(1, labels.view(-1, 1), 1)
        one_hot_labels = one_hot_labels * (1 - smoothing) + smoothing / num_classes

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, one_hot_labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    loss = train_loss / len(train_loader)

    return loss, acc

def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    loss = test_loss / len(test_loader)

    return loss, acc

# Train the model
train_losses, train_accs, valid_losses, valid_accs = [], [], [], []

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device, epoch)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    if ((epoch + 1) % 25 == 0):
      valid_loss, valid_acc = test(model, test_loader, criterion, device)
      valid_losses.append(valid_loss)   
      valid_accs.append(valid_acc)
      print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.2f}%')
    else:
      print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')

# Save the model
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch [1/250], Train Loss: 17.5633, Train Acc: 1.37%
Epoch [2/250], Train Loss: 6.2609, Train Acc: 2.25%
Epoch [3/250], Train Loss: 4.5755, Train Acc: 2.45%
Epoch [4/250], Train Loss: 4.5033, Train Acc: 2.75%
Epoch [5/250], Train Loss: 4.4257, Train Acc: 3.24%
Epoch [6/250], Train Loss: 4.3278, Train Acc: 4.41%
Epoch [7/250], Train Loss: 4.2299, Train Acc: 4.90%
Epoch [8/250], Train Loss: 4.1600, Train Acc: 6.47%
Epoch [9/250], Train Loss: 4.0447, Train Acc: 7.84%
Epoch [10/250], Train Loss: 3.9906, Train Acc: 7.84%
Epoch [11/250], Train Loss: 3.9296, Train Acc: 8.53%
Epoch [12/250], Train Loss: 3.9498, Train Acc: 9.22%
Epoch [13/250], Train Loss: 3.8849, Train Acc: 9.80%
Epoch [14/250], Train Loss: 3.8413, Train Acc: 9.12%
Epoch [15/250], Train Loss: 3.7667, Train Acc: 11.18%
Epoch [16/250], Train Loss: 3.7888, Train Acc: 11.67%
Epoch [17/250], Train Loss: 3.7390, Train Acc: 13.33%
Epoch [18/250], Train Loss: 3.7621, Train Acc: 11.27%
Epoch [19/250], Train Loss: 3.6817, Train Acc: 14.

In [3]:
# Did not save model, so load model
loaded_model = ConvNet()
loaded_model.load_state_dict(torch.load(PATH)) # it takes the loaded dictionary, not the path file itself
# Push to device
loaded_model.to(device)
loaded_model.eval()

# Evaluation, no gradient required
with torch.no_grad():
    n_correct = 0
    n_correct2 = 0
    n_samples = len(test_loader.dataset)

    for images, labels in test_loader:  # Iterate over test loader
        images = images.to(device)      # Push to GPU device
        labels = labels.to(device)
        outputs = model(images)

        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)    # Compare outputs with lables
        n_correct += (predicted == labels).sum().item() # Number of correct

        outputs2 = loaded_model(images)       # Same put loaded model
        _, predicted2 = torch.max(outputs2, 1)
        n_correct2 += (predicted2 == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the model: {acc} %')

    acc = 100.0 * n_correct2 / n_samples
    print(f'Accuracy of the loaded model: {acc} %')

Accuracy of the model: 42.706131078224104 %
Accuracy of the loaded model: 42.706131078224104 %
